## Run this notebook

You can launch this notbook using mybinder, by clicking the button below.

<a href="https://mybinder.org/v2/gh/NASA-IMPACT/veda-docs/HEAD?labpath=example-notebooks%2Fwfs.ipynb">
<img src="https://mybinder.org/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> 
</a>

## Approach

   1. Use `OWSLib` to determine what data is available and inspect the metadata
   2. Use `OWSLib` to filter and read the data
   3. Use `geopandas` and `folium` to analyze and plot the data

Note that the default examples environment is missing one requirement: `oswlib`. We can `pip install` that before we move on. 

In [37]:
!pip install OWSLib==0.28.1

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [38]:
from owslib.ogcapi.features import Features
import geopandas as gpd

## Look at the data that is availible through the OGC API

The datasets that are distributed throught the OGC API are organized into collections.
We can display the collections with the command: 

In [56]:
OGC_URL = "https://firenrt.delta-backend.com"

w = Features(url=OGC_URL)
w.feature_collections()

['public.eis_fire_perimeter',
 'public.eis_fire_newfirepix',
 'public.eis_fire_fireline',
 'public.st_subdivide',
 'public.st_hexagongrid',
 'public.st_squaregrid']

We will focus on the `public.eis_fire_fireline` collection and the `public.eis_fire_perimeter` collection. 

### Inspect the metatdata for public.eis_fire_perimeter collection

We can access information that drescribes the `public.eis_fire_perimeter`. 

In [57]:
perm = w.collection("public.eis_fire_perimeter")

We are particularly interested in the spatial and temporal extents of the data.

In [58]:
perm["extent"]

{'spatial': {'bbox': [[-124.47244468486822,
    24.073780394912994,
    -62.94003277624936,
    49.35277489239748]],
  'crs': 'http://www.opengis.net/def/crs/OGC/1.3/CRS84'},
 'temporal': {'interval': [['2023-03-03T00:00:00+00:00',
    '2023-03-23T00:00:00+00:00']],
  'trs': 'http://www.opengis.net/def/uom/ISO-8601/0/Gregorian'}}

In addition to getting metadata about the data we can access the queryable fields. Each of these fields will represent a column in our dataframe.

In [59]:
perm_q = w.collection_queryables('public.eis_fire_perimeter')
perm_q['properties']

{'wkb_geometry': {'$ref': 'https://geojson.org/schema/Geometry.json'},
 'ogc_fid': {'name': 'ogc_fid', 'type': 'number'},
 'n_pixels': {'name': 'n_pixels', 'type': 'number'},
 'n_newpixels': {'name': 'n_newpixels', 'type': 'number'},
 'farea': {'name': 'farea', 'type': 'number'},
 'fperim': {'name': 'fperim', 'type': 'number'},
 'flinelen': {'name': 'flinelen', 'type': 'number'},
 'duration': {'name': 'duration', 'type': 'number'},
 'pixden': {'name': 'pixden', 'type': 'number'},
 'meanfrp': {'name': 'meanfrp', 'type': 'number'},
 'isactive': {'name': 'isactive', 'type': 'number'},
 't': {'name': 't', 'type': 'string'},
 'fireid': {'name': 'fireid', 'type': 'number'}}

## Filter the data

It is always a good idea to do any data filtering as early as possible. In this example we know that we want the data for particular spatial and temporal extents. We can apply those and other filters using the `OWSLib` package.

In the below example we are:

 - choosing the `public.eis_fire_perimeter` collection 
 - subsetting it by space using the `bbox` parameter
 - subsetting it by time using the `datetime` parameter
 - filtering for fires over 5km^2 and over 2 days long using the `filter` parameter. The `filter` parameter lets us filter by the columns in 'public.eis_fire_perimeter' using SQL-style queries. 

NOTE: The `limit` parameter desginates the maximum number of objects the query will return. The default limit is 10, so if we want to all of the fire perimeters within certain conditions, we need to make sure that the limit is large.

In [60]:
perm_results = w.collection_items(
    'public.eis_fire_perimeter',   # name of the dataset we want
    bbox=['-106.8','24.5','-72.9','37.3'], # coodrinates of bounding box, 
    datetime=['2023-02-01T00:00:00+00:00/2023-03-05T12:00:00+00:00'], # date range
    limit=1000, # max number of items returned
    filter="farea>5 AND duration>2" # additional filters based on queryable fields 
)

The result is a dictionary containing all of the data and some summary fields. We can look at the keys to see what all is in there.

In [61]:
perm_results.keys()

dict_keys(['type', 'id', 'title', 'description', 'numberMatched', 'numberReturned', 'links', 'features'])

For instance you can check the total number of matched items and make sure that it is equal to the number of returned items. This is how you know that the `limit` you defined above is high enough.

In [62]:
perm_results['numberMatched'] == perm_results["numberReturned"]

True

You can also access the data directly in the browser or in an HTTP GET call using the constructed link.

In [63]:
perm_results['links'][1]['href']

'https://firenrt.delta-backend.com/collections/public.eis_fire_perimeter/items?bbox=-106.8%2C24.5%2C-72.9%2C37.3&datetime=2023-02-01T00%3A00%3A00%2B00%3A00%2F2023-03-05T12%3A00%3A00%2B00%3A00&limit=1000&filter=farea%3E5+AND+duration%3E2'

## Read data

In addition to all the summary fields, the `perm_results` dict contains all the data. We can pass the data into geopandas to make it easier to interact with.

In [64]:
df = gpd.GeoDataFrame.from_features(perm_results["features"])
df

,geometry,ogc_fid,n_pixels,n_newpixels,farea,fperim,flinelen,duration,pixden,meanfrp,isactive,t,fireid
0,"POLYGON ((-80.67714 28.47766, -80.67717 28.477...",89,45,0,9.977674,12.711405,0.0,3.0,4.510069,0.0,0,2023-03-03T12:00:00,22079
1,"POLYGON ((-89.11401 31.10912, -89.11387 31.109...",3610,12,0,5.154245,9.774314,0.0,6.0,2.328178,0.0,0,2023-03-03T12:00:00,20869
2,"MULTIPOLYGON (((-100.04571 29.37160, -100.0457...",10565,84,0,17.140797,40.696451,0.0,22.0,4.900589,0.0,0,2023-03-04T12:00:00,14603


## Explore data

We can quickly explore the data by setting the coordinate reference system (`crs`) and using `.explore()`

In [66]:
df = df.set_crs("EPSG:4326")
df.explore()

## Visualize Most Recent Fire Perimeters with Firelines

If we wanted to combine collections to make more informative analyses, we can use some of the same principles. 

First we'll get the queryable fields, and the extents:

In [67]:
fline_q = w.collection_queryables('public.eis_fire_fireline')
fline_collection = w.collection("public.eis_fire_fireline")
fline_q['properties']

{'wkb_geometry': {'$ref': 'https://geojson.org/schema/Geometry.json'},
 'ogc_fid': {'name': 'ogc_fid', 'type': 'number'},
 'fireid': {'name': 'fireid', 'type': 'number'},
 't': {'name': 't', 'type': 'string'},
 'mergeid': {'name': 'mergeid', 'type': 'number'}}

### Read
Then we'll use those fields to get most recent fire perimeters and fire lines. Here we are plotting the most recent time with both fire line data and fire perimeter data. 

In [70]:
most_recent_time = min(max(*fline_collection['extent']['temporal']['interval']), max(*perm['extent']['temporal']['interval']))
print("Most Recent Time =", most_recent_time)

## Get the most recent fire perimeters
perm_results = w.collection_items(
    "public.eis_fire_perimeter",   
    datetime=most_recent_time,
    limit=1000,   
)
perimeters = gpd.GeoDataFrame.from_features(perm_results['features'])

## Get the most recent fire lines
perimeter_ids = perimeters.fireid.unique()
perimeter_ids = ",".join(map(str, perimeter_ids)) 

fline_results = w.collection_items(
    "public.eis_fire_fireline",
    limit=1000,
    filter="fireid IN (" + perimeter_ids + ")" # only the fires from the fire perimeter query above
    
)
fline = gpd.GeoDataFrame.from_features(fline_results['features'])

Most Recent Time = 2023-03-23T00:00:00+00:00


### Visualize

In [71]:
perimeters = perimeters.set_crs("epsg:4326")
fline = fline.set_crs("epsg:4326")

m = perimeters.explore()
m = fline.explore(m=m, color="orange")
m

## Download Data

Downloading pre-filtered data may be useful for working locally, or for working with the data in GIS software. 


We can download the dataframe we made by writing it out into a shapefile or into a GeoJSON file.  

    perimeters.to_file('perimeters.shp') 
    perimeters.to_file('perimeters.geojson', driver='GeoJSON')

## About the Data


The Fire data shown is generated by the FEDs algorithm. The FEDs algorithm tracks fire movement and severity by ingesting observations from the VIIRS thermal sensors on the Suomi NPP and NOAA-20 satellites. This algorithm uses raw VIIRS observations to generate a polygon of the fire, locations of the active fire line, and estimates of fire mean Fire Radiative Power (FRP). The VIIRS sensors overpass at ~1:30 AM and PM local time, and provide estimates of fire evolution ~ every 12 hours. The data produced by this algorithm describe where fires are in space and how fires evolve through time. This CONUS-wide implementation of the FEDs algorithm is based on [Chen et al 2020’s](https://www.nature.com/articles/s41597-022-01343-0) algorithm for California.


The data produced by this algorithm is considered experimental. 


## Collection Information

**public.eis_fire_perimeter**

Perimeter of cumulative fire-area.

| Column      | Description | Unit      |
| ----------- | ----------- | ----------- |
| meanfrp      | Mean fire radiative power. The weighted sum of the fire radiative power detected at each new pixel, divided by the number of pixels.   If no new pixels are detected, meanfrp is set to zero. | MW/(pixel area)  |
| t   | Time of detection.       | Datetime. yyyy-mm-ddThh:mm:ss. Local time.  |
| fireid   | Fire ID. Unique for each fire. Matches fireid. |Numeric ID |
| pixden   |Number of pixels divided by area of perimeter.   | pixels/Km^2|
| duration  |Number of days since first observation of fire. Fires with a single observation have a duration of zero. |Days|
| flinelen   | Length of active fire line, based on new pixels. If no new pixels are detected, flinelen is set to zero.   |Km |
| fperim   | Length of fire perimeter.        | Km   |
| farea   | Area within fire perimeter.        |Km^2    |
| n_newpixels   | Number of pixels newly detected since last overpass.      |pixels  |
| n_pixels   | Number of pixel-detections in history of fire. | pixels   |
| isactive   | Have new fire pixels been detected in the last 5 days?     | Boolean |
| ogc_fid   |The ID used by the OGC API to sort perimeters.       | Numeric ID   |
| geometry  |The shape of the perimeter.    | Geometry    |


**public.eis_fire_fireline**

Active fire line as estimated by new VIIRS detections.

**public.eis_fire_newfirepix**

New pixel detections that inform the most recent time-step’s perimeter and fireline calculation.

| Column      | Description | Unit      |
| ----------- | ----------- | ----------- |
| fireid   | ID of fire pixel associated with.      | Datetime. yyyy-mm-ddThh:mm:ss. Local time.   |
| t   | Time of detection.   | Title       |
| mergeid  | ID used to connect pixels to perimeters. Matches fireid      | Numeric ID      |
| ogc_fid   |The ID used by the OGC API to sort pixels.      | Numeric ID     |
